In [1]:
from statPackage.DetectionStats import *
from statPackage.ParamProcessing import *
import multiprocessing


import os
import itertools
import matplotlib.pyplot as plt
import pickle
from zipfile import *

basePath = "D:/Downloads/BigTests/"
pickleName = "BigTests"

#basePath = "C:/Users/patrick/Downloads/driftTest/"
figurePath = "D:/Downloads/TestResults/"

X_VAL = ['detectionThreshold', 'detectionDistance']
X_VAL = ['validationThreshold', 'errorMultiplier', 'serverRecal']

IGNORE = ['movementPath', 'bombX', 'bombY', 'clusterMaxThresh']
ZIP = True

data = {}

In [2]:
def determineDifferences(basePath, runs):
    params = {}
    for r in runs:
    # for i in range(500):
        p = getParameters("%s%s" % (basePath, r))
        print(r)
        #print(p['serverRecal'])
        for k in p.keys():
            if 'file' not in k and 'File' not in k and k not in IGNORE:
                if k in params:
                    params[k].add(p[k])
                else:
                    params[k] = set()
                    params[k].add(p[k])

    unique = {}
    for k in params.keys():
        if len(params[k]) > 1:
            unique[k] = list(sorted(params[k]))
    #print(unique)
    return unique

def runner(queue):
    while True:
        job = queue.get()
        print("%d\\%d" % (job[1], job[2]))
        #print(job)
        command = "./simulator/Simulation "+' '.join(job[0])
        print(command)
        #os.system(command + " 1>/dev/null")
        queue.task_done()

In [3]:
def dataStorage(wq, order, variation, total):

    i = 0
    failed = 0
    pickleOpen = False
    data = {}
    waiting = 0
    while True:
        job = wq.get()
        if not pickleOpen:
            try:
                print("opening")
                with open('%s.pickle' % (pickleName), 'rb') as handle:
                    data = pickle.load(handle)# protocol=pickle.HIGHEST_PROTOCOL)
                    data = data['data']
                    handle.close()
                    pickleOpen = True
            except:
                pass
        run = job[0]
        det = job[1]
        k = job[2]

        if k in data:
            data[k].append(run)
        else:
            data[k] = [run]
        waiting += 1

        dat = {'data': data, 'order': order, 'var': variation}
        if waiting > 200 or wq.qsize() < 2:
            try:
                f = open('%s.pickle' % (pickleName), 'wb')
                pickle.dump(dat, f)
                f.close()
                waiting = 0
                pickleOpen = False
                #fail = False
            except:
                failed += 1
                pass
        else:
            print(waiting, wq.qsize())
        i += 1
        print("Total %d/%d Failed %d/%d" % (i, total, failed, i))
        #with open('driftExploreBomb.pickle', 'wb') as handle:
        #    pickle.dump(data, handle)# protocol=pickle.HIGHEST_PROTOCOL)
        #    handle.close()
        wq.task_done()
        if wq.qsize() <= 0:
            break

    return data

In [4]:
def getBatteryStats(zipPath, zipName):
    zf = ZipFile(zipPath + zipName)
    f = zf.open("%s%s" % (zipName.split(".zip")[0], "-batteryusage.txt"))
    percentAliveOverTime = []

    for line in f:
        line = line.decode("utf-8")
        if "Iteration" in line:
            line = line.split(",")[1].rstrip()
            percentAlive = float(line)
            percentAliveOverTime += [percentAlive]
        elif "Average Remaining Battery" in line:
            averageRemaining = float(line.split(":")[1].rstrip())
        elif "Total Dead Nodes" in line:
            fraction = line.split(":")[1].rstrip()
            deadNodes = float(fraction.split("/")[0].rstrip())
            totalNodes = float(fraction.split("/")[1].rstrip())
            percentDead = deadNodes/totalNodes

    return [averageRemaining, percentDead, percentAliveOverTime]

In [5]:
def getClusterStats(zipPath, zipName):
    zf = ZipFile(zipPath + zipName)
    f = zf.open("%s%s" % (zipName.split(".zip")[0], "-clusterStats.txt"))
    clusterStats = []

    for line in f:
        line = line.decode("utf-8")
        if 'Lost Readings' in line:
            fraction = line.split(':')[1].rstrip()
            top = float(fraction.split('/')[0].rstrip())
            bottom = float(fraction.split('/')[1].rstrip())
            clusterStats += [top/bottom]
        elif line.strip():
            clusterStats += [int(line.split(':')[1].rstrip())]

    return clusterStats

In [6]:
uniqueRuns = set()
for file in os.listdir(basePath):
    uniqueRuns.add(file.split('-')[0])
uniqueRuns = list(uniqueRuns)#[:100]

In [8]:
shiftingParameters = determineDifferences(basePath, uniqueRuns)
print(shiftingParameters)

Log_18324.zip
Log_5494.zip
Log_732.zip
Log_17547.zip
Log_22604.zip
Log_8826.zip
Log_3099.zip
Log_5371.zip
Log_20404.zip
Log_11218.zip
Log_2603.zip
Log_20789.zip
Log_787.zip
Log_10815.zip
Log_20721.zip
Log_23563.zip
Log_11215.zip
Log_14782.zip
Log_21957.zip
Log_23865.zip
Log_17068.zip
Log_7789.zip
Log_18924.zip
Log_14989.zip
Log_4274.zip
Log_57.zip
Log_8354.zip
Log_6884.zip
Log_193.zip
Log_15575.zip
Log_613.zip
Log_10292.zip
Log_16929.zip
Log_10656.zip
Log_21751.zip
Log_15850.zip
Log_1797.zip
Log_22388.zip
Log_17592.zip
Log_11806.zip
Log_20552.zip
Log_6417.zip
Log_8220.zip
Log_19933.zip
Log_3236.zip
Log_18088.zip
Log_15822.zip
Log_8248.zip
Log_14702.zip
Log_21599.zip
Log_6993.zip
Log_14760.zip
Log_17578.zip
Log_6686.zip
Log_8977.zip
Log_3010.zip
Log_5845.zip
Log_13627.zip
Log_14525.zip
Log_8443.zip
Log_11966.zip
Log_11004.zip
Log_997.zip
Log_2055.zip
Log_12411.zip
Log_1321.zip
Log_16056.zip
Log_10198.zip
Log_12889.zip
Log_19553.zip
Log_7728.zip
Log_21921.zip
Log_19847.zip
Log_16389.zip


Log_6906.zip
Log_17219.zip
Log_2483.zip
Log_18315.zip
Log_12909.zip
Log_22442.zip
Log_1705.zip
Log_16850.zip
Log_10998.zip
Log_16574.zip
Log_6528.zip
Log_13678.zip
Log_18437.zip
Log_22244.zip
Log_6709.zip
Log_9633.zip
Log_9978.zip
Log_5319.zip
Log_23334.zip
Log_21867.zip
Log_9083.zip
Log_2580.zip
Log_11549.zip
Log_23656.zip
Log_5805.zip
Log_2393.zip
Log_13098.zip
Log_7466.zip
Log_11358.zip
Log_13615.zip
Log_7151.zip
Log_12412.zip
Log_15390.zip
Log_6572.zip
Log_19798.zip
Log_17399.zip
Log_11891.zip
Log_7702.zip
Log_9672.zip
Log_17973.zip
Log_12047.zip
Log_17695.zip
Log_3543.zip
Log_16593.zip
Log_7699.zip
Log_14358.zip
Log_13006.zip
Log_1445.zip
Log_4152.zip
Log_2665.zip
Log_5943.zip
Log_15430.zip
Log_19868.zip
Log_15689.zip
Log_13177.zip
Log_10689.zip
Log_616.zip
Log_17104.zip
Log_18524.zip
Log_7568.zip
Log_13723.zip
Log_7006.zip
Log_12096.zip
Log_3144.zip
Log_23036.zip
Log_5395.zip
Log_3726.zip
Log_858.zip
Log_16166.zip
Log_21408.zip
Log_5468.zip
Log_17334.zip
Log_3544.zip
Log_10881.zi

Log_7974.zip
Log_12203.zip
Log_13781.zip
Log_5817.zip
Log_5790.zip
Log_14253.zip
Log_663.zip
Log_426.zip
Log_24289.zip
Log_5893.zip
Log_3304.zip
Log_13108.zip
Log_17734.zip
Log_10167.zip
Log_7370.zip
Log_4861.zip
Log_24060.zip
Log_22769.zip
Log_9737.zip
Log_2689.zip
Log_528.zip
Log_4705.zip
Log_11283.zip
Log_798.zip
Log_23621.zip
Log_17367.zip
Log_22087.zip
Log_9864.zip
Log_1354.zip
Log_15372.zip
Log_19360.zip
Log_7701.zip
Log_16525.zip
Log_17659.zip
Log_18304.zip
Log_2658.zip
Log_3870.zip
Log_19338.zip
Log_18472.zip
Log_8458.zip
Log_23482.zip
Log_21295.zip
Log_12481.zip
Log_17543.zip
Log_21869.zip
Log_20601.zip
Log_20513.zip
Log_22339.zip
Log_8031.zip
Log_13485.zip
Log_5994.zip
Log_19790.zip
Log_22827.zip
Log_20863.zip
Log_3511.zip
Log_12990.zip
Log_1181.zip
Log_1326.zip
Log_9078.zip
Log_12213.zip
Log_16165.zip
Log_18895.zip
Log_9836.zip
Log_20528.zip
Log_5945.zip
Log_22648.zip
Log_7579.zip
Log_15307.zip
Log_19172.zip
Log_23006.zip
Log_5542.zip
Log_13219.zip
Log_22562.zip
Log_8806.zip

Log_5457.zip
Log_12014.zip
Log_3061.zip
Log_7295.zip
Log_20740.zip
Log_11759.zip
Log_4969.zip
Log_18684.zip
Log_16275.zip
Log_17842.zip
Log_768.zip
Log_7052.zip
Log_4621.zip
Log_4111.zip
Log_4871.zip
Log_18619.zip
Log_16406.zip
Log_8291.zip
Log_3897.zip
Log_19670.zip
Log_19990.zip
Log_198.zip
Log_9502.zip
Log_14014.zip
Log_12147.zip
Log_14934.zip
Log_673.zip
Log_5218.zip
Log_20881.zip
Log_4781.zip
Log_12508.zip
Log_20333.zip
Log_8805.zip
Log_18573.zip
Log_4530.zip
Log_21799.zip
Log_18100.zip
Log_22597.zip
Log_22727.zip
Log_11708.zip
Log_4544.zip
Log_8049.zip
Log_18794.zip
Log_11007.zip
Log_16257.zip
Log_18844.zip
Log_629.zip
Log_18018.zip
Log_19226.zip
Log_15136.zip
Log_12474.zip
Log_10164.zip
Log_19735.zip
Log_7285.zip
Log_7069.zip
Log_1412.zip
Log_13802.zip
Log_9248.zip
Log_4143.zip
Log_17941.zip
Log_23300.zip
Log_6082.zip
Log_5208.zip
Log_11104.zip
Log_22784.zip
Log_17653.zip
Log_5772.zip
Log_20826.zip
Log_6347.zip
Log_19421.zip
Log_14417.zip
Log_9516.zip
Log_3598.zip
Log_8756.zip
L

Log_19876.zip
Log_15966.zip
Log_8748.zip
Log_18102.zip
Log_21782.zip
Log_8308.zip
Log_22267.zip
Log_6184.zip
Log_10156.zip
Log_20317.zip
Log_2106.zip
Log_20862.zip
Log_5275.zip
Log_8614.zip
Log_5651.zip
Log_13223.zip
Log_4726.zip
Log_8817.zip
Log_21254.zip
Log_15657.zip
Log_11705.zip
Log_2449.zip
Log_1113.zip
Log_17927.zip
Log_12869.zip
Log_11874.zip
Log_9127.zip
Log_458.zip
Log_822.zip
Log_15879.zip
Log_2202.zip
Log_18215.zip
Log_8042.zip
Log_4562.zip
Log_13381.zip
Log_3799.zip
Log_6565.zip
Log_15557.zip
Log_22464.zip
Log_6770.zip
Log_13479.zip
Log_18519.zip
Log_7697.zip
Log_8782.zip
Log_10745.zip
Log_272.zip
Log_23519.zip
Log_13368.zip
Log_23559.zip
Log_17048.zip
Log_5541.zip
Log_6161.zip
Log_7536.zip
Log_16437.zip
Log_14037.zip
Log_15190.zip
Log_11713.zip
Log_10961.zip
Log_3974.zip
Log_17981.zip
Log_4878.zip
Log_7969.zip
Log_6110.zip
Log_10939.zip
Log_5608.zip
Log_22892.zip
Log_453.zip
Log_1377.zip
Log_1544.zip
Log_18162.zip
Log_530.zip
Log_4253.zip
Log_6776.zip
Log_202.zip
Log_1891

...

Log_13076.zip
Log_16170.zip
Log_13878.zip
Log_7665.zip
Log_1363.zip
Log_14108.zip
Log_22893.zip
Log_1287.zip
Log_19232.zip
Log_13491.zip
Log_13071.zip
Log_4217.zip
Log_4680.zip
Log_4374.zip
Log_2971.zip
Log_13501.zip
Log_13110.zip
Log_604.zip
Log_10850.zip
Log_14992.zip
Log_6602.zip
Log_17733.zip
Log_23698.zip
Log_15261.zip
Log_16653.zip
Log_19271.zip
Log_15040.zip
Log_4025.zip
Log_15302.zip
Log_22383.zip
Log_12975.zip
Log_1845.zip
Log_21143.zip
Log_20667.zip
Log_12321.zip
Log_9872.zip
Log_11682.zip
Log_11784.zip
Log_18427.zip
Log_8939.zip
Log_12693.zip
Log_4874.zip
Log_7950.zip
Log_9971.zip
Log_21357.zip
Log_11817.zip
Log_9675.zip
Log_11528.zip
Log_24107.zip
Log_17084.zip
Log_7108.zip
Log_23514.zip
Log_19564.zip
Log_11802.zip
Log_21216.zip
Log_7453.zip
Log_16153.zip
Log_15526.zip
Log_19091.zip
Log_9128.zip
Log_7497.zip
Log_10633.zip
Log_3947.zip
Log_1488.zip
Log_5770.zip
Log_16061.zip
Log_19397.zip
Log_21575.zip
Log_10400.zip
Log_21399.zip
Log_5330.zip
Log_9182.zip
Log_22365.zip
Log_1

Log_19802.zip
Log_2466.zip
Log_9451.zip
Log_7821.zip
Log_266.zip
Log_2618.zip
Log_5398.zip
Log_19516.zip
Log_15286.zip
Log_13063.zip
Log_1394.zip
Log_18870.zip
Log_3558.zip
Log_5913.zip
Log_14376.zip
Log_19759.zip
Log_2147.zip
Log_22330.zip
Log_4927.zip
Log_4866.zip
Log_7574.zip
Log_23119.zip
Log_1088.zip
Log_10097.zip
Log_2574.zip
Log_4985.zip
Log_14815.zip
Log_15970.zip
Log_21239.zip
Log_9882.zip
Log_5606.zip
Log_21006.zip
Log_22911.zip
Log_18737.zip
Log_12299.zip
Log_23297.zip
Log_8253.zip
Log_15297.zip
Log_9409.zip
Log_18074.zip
Log_20180.zip
Log_6848.zip
Log_4757.zip
Log_13318.zip
Log_7677.zip
Log_13444.zip
Log_18165.zip
Log_15510.zip
Log_11923.zip
Log_19166.zip
Log_4189.zip
Log_6614.zip
Log_1346.zip
Log_23122.zip
Log_17298.zip
Log_23758.zip
Log_3129.zip
Log_21569.zip
Log_6582.zip
Log_15624.zip
Log_7920.zip
Log_13947.zip
Log_1023.zip
Log_18554.zip
Log_640.zip
Log_16037.zip
Log_24226.zip
Log_16132.zip
Log_5673.zip
Log_448.zip
Log_3619.zip
Log_4433.zip
Log_20211.zip
Log_14391.zip
Lo

Log_3029.zip
Log_14676.zip
Log_23962.zip
Log_6278.zip
Log_11103.zip
Log_9942.zip
Log_14341.zip
Log_22049.zip
Log_23347.zip
Log_15498.zip
Log_18610.zip
Log_12957.zip
Log_18551.zip
Log_9701.zip
Log_13438.zip
Log_14858.zip
Log_1928.zip
Log_4147.zip
Log_10954.zip
Log_5866.zip
Log_4741.zip
Log_15379.zip
Log_9148.zip
Log_17748.zip
Log_3187.zip
Log_9447.zip
Log_10786.zip
Log_4783.zip
Log_10173.zip
Log_10544.zip
Log_22859.zip
Log_18460.zip
Log_19195.zip
Log_15250.zip
Log_1814.zip
Log_1526.zip
Log_22291.zip
Log_9273.zip
Log_21549.zip
Log_2487.zip
Log_16553.zip
Log_13159.zip
Log_21659.zip
Log_6900.zip
Log_5781.zip
Log_20127.zip
Log_16122.zip
Log_18247.zip
Log_14284.zip
Log_11098.zip
Log_20746.zip
Log_15406.zip
Log_622.zip
Log_20608.zip
Log_4257.zip
Log_18284.zip
Log_8761.zip
Log_7828.zip
Log_8902.zip
Log_2512.zip
Log_4554.zip
Log_9172.zip
Log_3921.zip
Log_15070.zip
Log_12519.zip
Log_7479.zip
Log_15561.zip
Log_16453.zip
Log_16559.zip
Log_5797.zip
Log_2532.zip
Log_23996.zip
Log_9325.zip
Log_6759.z

Log_3809.zip
Log_5750.zip
Log_9411.zip
Log_19907.zip
Log_12792.zip
Log_16894.zip
Log_17746.zip
Log_9761.zip
Log_12091.zip
Log_19941.zip
Log_225.zip
Log_3471.zip
Log_5759.zip
Log_20294.zip
Log_23733.zip
Log_18285.zip
Log_3519.zip
Log_23013.zip
Log_18814.zip
Log_16560.zip
Log_20722.zip
Log_23377.zip
Log_5718.zip
Log_11691.zip
Log_3823.zip
Log_12409.zip
Log_9966.zip
Log_16954.zip
Log_3930.zip
Log_21145.zip
Log_7808.zip
Log_16359.zip
Log_10860.zip
Log_8123.zip
Log_5163.zip
Log_20727.zip
Log_23130.zip
Log_4585.zip
Log_4640.zip
Log_9758.zip
Log_5341.zip
Log_17507.zip
Log_14353.zip
Log_9591.zip
Log_16220.zip
Log_17514.zip
Log_18920.zip
Log_19249.zip
Log_6758.zip
Log_7860.zip
Log_13554.zip
Log_510.zip
Log_156.zip
Log_14522.zip
Log_20300.zip
Log_6311.zip
Log_13043.zip
Log_24335.zip
Log_19975.zip
Log_18711.zip
Log_8044.zip
Log_14655.zip
Log_19494.zip
Log_4714.zip
Log_11941.zip
Log_6250.zip
Log_5828.zip
Log_22561.zip
Log_11327.zip
Log_14966.zip
Log_7680.zip
Log_20905.zip
Log_17568.zip
Log_21888.z

Log_17023.zip
Log_6577.zip
Log_1891.zip
Log_1278.zip
Log_13401.zip
Log_22959.zip
Log_4349.zip
Log_23243.zip
Log_4233.zip
Log_7197.zip
Log_15893.zip
Log_11082.zip
Log_14898.zip
Log_13046.zip
Log_18622.zip
Log_12818.zip
Log_1435.zip
Log_11734.zip
Log_9156.zip
Log_5100.zip
Log_21535.zip
Log_8352.zip
Log_11890.zip
Log_22759.zip
Log_2501.zip
Log_3971.zip
Log_9766.zip
Log_11099.zip
Log_1124.zip
Log_4539.zip
Log_13030.zip
Log_22719.zip
Log_15746.zip
Log_10056.zip
Log_5411.zip
Log_11997.zip
Log_23667.zip
Log_10690.zip
Log_14572.zip
Log_8960.zip
Log_14769.zip
Log_16033.zip
Log_19399.zip
Log_8891.zip
Log_7761.zip
Log_9670.zip
Log_9755.zip
Log_12809.zip
Log_11974.zip
Log_2685.zip
Log_8496.zip
Log_9418.zip
Log_22145.zip
Log_21765.zip
Log_20386.zip
Log_14176.zip
Log_14267.zip
Log_14910.zip
Log_13207.zip
Log_4022.zip
Log_9771.zip
Log_10489.zip
Log_13092.zip
Log_17883.zip
Log_3086.zip
Log_19464.zip
Log_10272.zip
Log_10422.zip
Log_14060.zip
Log_11188.zip
Log_24039.zip
Log_11778.zip
Log_8926.zip
Log_12

In [9]:
vals = []
order = []
for k in shiftingParameters.keys():
    order.append(k)
    vals.append(shiftingParameters[k])
    
print(order)
xx = list(itertools.product(*vals))
print(xx)

['totalNodes', 'clusterMinThresh', 'clusteringOn', 'globalRecluster', 'localRecluster', 'reclusterPeriod', 'reclusterThreshold', 'bluetoothLossPercentage', 'wifiLossPercentage']
[(500, 0, 'false', 'false', 0, 5, 0.025, 0, 2e-05), (500, 0, 'false', 'false', 0, 5, 0.025, 0, 5e-05), (500, 0, 'false', 'false', 0, 5, 0.025, 1e-06, 2e-05), (500, 0, 'false', 'false', 0, 5, 0.025, 1e-06, 5e-05), (500, 0, 'false', 'false', 0, 5, 0.025, 2e-06, 2e-05), (500, 0, 'false', 'false', 0, 5, 0.025, 2e-06, 5e-05), (500, 0, 'false', 'false', 0, 5, 0.025, 5e-06, 2e-05), (500, 0, 'false', 'false', 0, 5, 0.025, 5e-06, 5e-05), (500, 0, 'false', 'false', 0, 5, 0.025, 2e-05, 2e-05), (500, 0, 'false', 'false', 0, 5, 0.025, 2e-05, 5e-05), (500, 0, 'false', 'false', 0, 5, 0.05, 0, 2e-05), (500, 0, 'false', 'false', 0, 5, 0.05, 0, 5e-05), (500, 0, 'false', 'false', 0, 5, 0.05, 1e-06, 2e-05), (500, 0, 'false', 'false', 0, 5, 0.05, 1e-06, 5e-05), (500, 0, 'false', 'false', 0, 5, 0.05, 2e-06, 2e-05), (500, 0, 'false',

In [ ]:
from StatisticsGenerator import generateData

if __name__ == '__main__':
    print('start')
    m = multiprocessing
    rq = m.JoinableQueue()
    wq = m.JoinableQueue()

    for run in uniqueRuns:
        rq.put([run, order])
    # for i in range(500):
    #     rq.put([uniqueRuns[i], order])

    p = multiprocessing.Pool(6, generateData, (rq,wq,), maxtasksperchild=3)

    rq.join()

    print('here')

start


In [ ]:
dataStorage(wq, order, shiftingParameters, len(uniqueRuns))